<a href="https://colab.research.google.com/github/passion1014/rag/blob/main/LCEL_Test_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_core
!pip install langchain_openai

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import numexpr

In [ ]:
prompt_template = ChatPromptTemplate.from_template(
    "For {query}, write only the mathematical expression suitable for numexpr.evaluate().")
chain = prompt_template | ChatOpenAI() | StrOutputParser() | numexpr.evaluate
chain.invoke({'query' : 'What is 13 raised to the .3432 power?'})

array(2.41160046)

In [ ]:
numexpr.evaluate('13**.3432')

array(2.41160046)